# Covid-19 Daily Tracking


In [162]:
import sys
import os.path
import requests
import json
from datetime import datetime 
import pandas as pd

COUNTRY_LOC = 'Malaysia'
API_URL = 'https://api.covid19api.com/total/country/' + COUNTRY_LOC
COVID_LOCAL = '../data/covid_19_' + COUNTRY_LOC + '.json'

start_date = '2021-02-01'
today_date = datetime.today()
url_param = {'from': start_date, 'to': str(today_date)[:10]}

try:
    api_request = requests.get(API_URL, params=url_param, timeout=30)

    if os.path.isfile(COVID_LOCAL):
        print('Exist')
        t = os.path.getmtime(COVID_LOCAL)
        file_date = (str(datetime.fromtimestamp(t)))[:10]

        if file_date < str(today_date)[:10]:
            if api_request.ok:
                print('Outdated File. Updating new data from web.....')
                with open(COVID_LOCAL, "wb") as file:
                    file.write(api_request.content) 
                df = pd.read_json(COVID_LOCAL)
                df.set_index('Date', inplace=True)
            else:
                print('Error: ', api_request.status_code)
                print('Use existing file.....')
                df = pd.read_json(COVID_LOCAL)
                df.set_index('Date', inplace=True)
        else:
            print('Retrieving from current file....')
            df = pd.read_json(COVID_LOCAL)
            df.set_index('Date', inplace=True)
        
    else:
        if api_request.ok:
            print('Pulling data from web to new file....')
            with open(COVID_LOCAL, "wb") as file:
                file.write(api_request.content) 
            df = pd.read_json(COVID_LOCAL)
            df.set_index('Date', inplace=True)
        else:
            print('Error: ', api_request.status_code)
            print('No Data File exist....quiting')
            sys.exit(0)

except requests.exceptions.RequestException as e:
    print('Server Request Failed :', e)
    print('using existing file ....')
    df = pd.read_json(COVID_LOCAL)
    df.set_index('Date', inplace=True)




Exist
Retrieving from current file....


In [163]:
df

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active
Date,,,,,,,,,,,
2020-01-22 00:00:00+00:00,Malaysia,,,,,0,0,0,0,0,0
2020-01-23 00:00:00+00:00,Malaysia,,,,,0,0,0,0,0,0
2020-01-24 00:00:00+00:00,Malaysia,,,,,0,0,0,0,0,0
2020-01-25 00:00:00+00:00,Malaysia,,,,,0,0,3,0,0,3
2020-01-26 00:00:00+00:00,Malaysia,,,,,0,0,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31 00:00:00+00:00,Malaysia,,,,,0,0,8976,125,8644,207
2020-08-01 00:00:00+00:00,Malaysia,,,,,0,0,8985,125,8647,213
2020-08-02 00:00:00+00:00,Malaysia,,,,,0,0,8999,125,8664,210


In [164]:
# remove unwanted Columns
df.drop(columns=['Country', 'CountryCode', 'Province', 'City', 'CityCode', 'Lat', "Lon"], inplace=True)

In [165]:
# rename columns date to strip of time info
for item in df.index:
    df.rename(index={item: str(item)[:10]}, inplace=True)
df

,Confirmed,Deaths,Recovered,Active
Date,,,,
2020-01-22,0,0,0,0
2020-01-23,0,0,0,0
2020-01-24,0,0,0,0
2020-01-25,3,0,0,3
2020-01-26,4,0,0,4
...,...,...,...,...
2020-07-31,8976,125,8644,207
2020-08-01,8985,125,8647,213
2020-08-02,8999,125,8664,210


In [166]:
# Population by Country data pulled from UN [source: https://population.un.org/wpp/Download/Standard/Population/] edited to conform to country's name
pop_df = pd.read_csv('../data/population.csv')
"""
for index, item in enumerate(pop_df['Pop.(\'000)']):
    # strip empty spaces and convert to integer (in millions)
    pop_df.loc[index, 'Pop.(\'000)'] = int(item.replace(' ', ''))
    pop_df.loc[index, 'Pop.(\'000)'] = pop_df.loc[index, 'Pop.(\'000)'] * 1000 

    # rename Country's name to conform to covid's dataframe
    pop_df.loc[index, 'Country'] = pop_df.loc[index, 'Country'].lower().replace(' ', '-')
"""
pop_df['Country'] = pop_df['Country'].str.replace(' ', '-') #.str.lower()
pop_df['Pop.(\'000)'] = pop_df['Pop.(\'000)'].str.replace(' ', '').astype(int) \
 * 1000 
pop_df.set_index('Country', inplace=True)
pop_df

,Pop.('000)
Country,
Burundi,11891000
Comoros,870000
Djibouti,988000
Eritrea,3546000
Ethiopia,114964000
...,...
Bermuda,62000
Canada,37742000
Greenland,57000


## Data Crunching to form new dataset


In [167]:
daily_cases = []
daily_deaths = []
daily_recovered = []

""" OBSOLETE - REFACTOR

def append_rate_data(index):
  
    mortality_rate.append(round((df.loc[df.index[index], 'Deaths'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    recovered_rate.append(round((df.loc[df.index[index], 'Recovered'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    active_rate.append(round((df.loc[df.index[index], 'Active'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    cases_per_million_pop.append(round((df.loc[df.index[index], 'Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)']) * 1000000, 0))    
"""
      
for index, item in enumerate(df.index):
    if item == df.index[0]:
        new_case, new_death, new_recovered = df.loc[df.index[0], ['Confirmed', 'Deaths', 'Recovered']] 
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    else:
        new_case, new_death, new_recovered = df.loc[df.index[index], ['Confirmed', 'Deaths', 'Recovered']] \
                                            - df.loc[df.index[index - 1], ['Confirmed', 'Deaths', 'Recovered']]
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    df.loc[item, ['Daily Cases', 'Daily Deaths', 'Daily Recovered']] \
                  = daily_cases[index], daily_deaths[index], daily_recovered[index]

# release unwanted objects from memory
del daily_cases, daily_deaths, daily_recovered


df['Mortality Rates'] = round(df['Deaths'].divide(df ['Confirmed']).fillna(0.0), 4) * 100
df['Recovered Rates'] = round(df['Recovered'].divide(df ['Confirmed']).fillna(0.0), 4) * 100
df['Active Rates'] = round(df['Active'].divide(df ['Confirmed']).fillna(0.0), 4) * 100
df['Cases per 1mil pop'] = ((df['Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)']) * 1000000).astype(int)
df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']] = df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].astype(int)


df


,Confirmed,Deaths,Recovered,Active,Daily Cases,Daily Deaths,Daily Recovered,Mortality Rates,Recovered Rates,Active Rates,Cases per 1mil pop
Date,,,,,,,,,,,
2020-01-22,0,0,0,0,0,0,0,0.00,0.00,0.00,0
2020-01-23,0,0,0,0,0,0,0,0.00,0.00,0.00,0
2020-01-24,0,0,0,0,0,0,0,0.00,0.00,0.00,0
2020-01-25,3,0,0,3,3,0,0,0.00,0.00,100.00,0
2020-01-26,4,0,0,4,1,0,0,0.00,0.00,100.00,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31,8976,125,8644,207,12,1,27,1.39,96.30,2.31,277
2020-08-01,8985,125,8647,213,9,0,3,1.39,96.24,2.37,277
2020-08-02,8999,125,8664,210,14,0,17,1.39,96.28,2.33,278


In [168]:
# replace NaN to zero
#df.fillna(0.0, inplace=True)
df.tail(7)

,Confirmed,Deaths,Recovered,Active,Daily Cases,Daily Deaths,Daily Recovered,Mortality Rates,Recovered Rates,Active Rates,Cases per 1mil pop
Date,,,,,,,,,,,
2020-07-29,8956,124,8612,220,13,0,5,1.38,96.16,2.46,276
2020-07-30,8964,124,8617,223,8,0,5,1.38,96.13,2.49,276
2020-07-31,8976,125,8644,207,12,1,27,1.39,96.30,2.31,277
2020-08-01,8985,125,8647,213,9,0,3,1.39,96.24,2.37,277
2020-08-02,8999,125,8664,210,14,0,17,1.39,96.28,2.33,278
2020-08-03,9001,125,8668,208,2,0,4,1.39,96.30,2.31,278
2020-08-04,9002,125,8684,193,1,0,16,1.39,96.47,2.14,278


In [169]:
# Max Daily Record
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].max(), 2)

Daily Cases        277
Daily Deaths         8
Daily Recovered    333
dtype: int64

In [170]:
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].mean(), 2)

Daily Cases        45.93
Daily Deaths        0.64
Daily Recovered    44.31
dtype: float64

In [171]:
last_entry_date = df.index[len(df.index) - 1]
df.loc[last_entry_date]

Confirmed             9002.00
Deaths                 125.00
Recovered             8684.00
Active                 193.00
Daily Cases              1.00
Daily Deaths             0.00
Daily Recovered         16.00
Mortality Rates          1.39
Recovered Rates         96.47
Active Rates             2.14
Cases per 1mil pop     278.00
Name: 2020-08-04, dtype: float64